In [11]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [12]:
from itertools import islice
import time
from random import randint
import random
from typing import List

In [13]:
from Surfer import GetBrowser,RedditFeed,Feed,ChatGPT,ChatGPTWebsite
from Surfer.ScriptRunner import ScriptRunner
from files import *
from Stegasus import stegasus
from Bot import *
from Chatter import prompts
from Surfer.util import uuid

In [14]:
# Example usage
ex = "png"
file_path = f'micro_rick_roll.{ex}'
data = ''
for chunk in read_file_in_chunks(file_path):
    data += chunk
data

'100010010101000001001110010001110000110100001010000110100000101000000000000000000000000000001101010010010100100001000100010100100000000000000000000000000000011100000000000000000000000000000111000010000000011000000000000000000000000011000100010100100101011111010011000000000000000000000000000010010111000001001000010110010111001100000000000000000000001111011000000000000000000000000011110110000000000101101110010001110110001100110101000000000000000000000000110101100100100101000100010000010101010000001000010110110000000111001011000000000011010011111111000000011010101110101001101011011111111111111100111110001110110100000000111101101100110111010011000000001110100111010101111000000000000000000111001011010010100100000000000101000011000100101011000000001111111111111111111111110000000000000001101100111011011010110110111111111111111011101100111011000000000011100110101110001100010100000000000010000000000100001010000000001111100100100101000110000000000000001001001000010001100000000000111111111111111

In [15]:
# KB
print(len(data))
data = data[:2**10]
print(len(data))

2336
1024


In [16]:
cache = {}

In [90]:
class Encoder:
	def __init__(self,data,dataLog,encodedChatLog,identity=False):
		self.data = data
		self.rem = data
		self.dataLog = dataLog
		self.encodedChatLog = encodedChatLog
		self.dataLog.append('')
		self.sending_pure_padding = False
		self.identity = identity
		self.size = 0

	def done(self):
		self.size += 1
		return self.sending_pure_padding if not self.identity else self.size <= 10

	def __call__(self,text) -> str:
		if self.identity:
			return text
		# sending pure padding
		self.sending_pure_padding = len(self.rem) < 499
		a, self.rem = stegasus.StegasusEncode(text,self.rem)
		self.encodedChatLog.append(a)
		self.dataLog.append(data[len(self.dataLog[-1]):-len(self.rem)])
		return a


In [91]:
# import importlib
# importlib.reload(prompts)

# backup_post = [p for p in cache['posts'] if not (p['title'] is not None and p['title'] != "" and p['excerpt'] is not None and p['excerpt'] != "")][0]
# a = prompts.inChatReplyPrompt(Message(Person('alice','nice',23,'female','Boston','alice@gmail.com'),'sup dog'),backup_post)
# print(a)

In [92]:
print(list(cache.keys()))
# del cache['data']
# del cache['driver']
# del cache['RedditFeed']
# del cache['posts']
# del cache['ChatGPTWebsite']
# del cache['encodedChatLog']
# del cache['chatLog']
# del cache['promptLog']
# del cache['dataLog']
# del cache['aliceChat']
# del cache['bobChat']'
print(list(cache.keys()))

['data', 'driver', 'RedditFeed', 'posts', 'ChatGPTWebsite', 'encodedChatLog', 'chatLog', 'promptLog', 'dataLog', 'aliceChat', 'bobChat']
['data', 'driver', 'RedditFeed', 'posts', 'ChatGPTWebsite', 'encodedChatLog', 'chatLog', 'promptLog', 'dataLog', 'aliceChat', 'bobChat']


In [97]:
def openChannelAndSend(data:str
, alice = Person('alice','nice',23,'female','Boston','alice@gmail.com')
, bob = Person('bob','wick',29,'male','Boston','bob@gmail.com')):
	data += '1' + ('0' * 500)
	cache['data'] = data
	if 'driver' in cache:
		driver = cache['driver']
	else:
		cache['driver'] = GetBrowser()
		driver = cache['driver']
	driver.implicitly_wait(2)
	cache['RedditFeed']= cache.get('RedditFeed',RedditFeed(driver,skip=False,subreddit='unpopularopinion',newWindow=False))
	rf = cache['RedditFeed'] 
	# rf = RedditFeed(driver,skip=True)
	posts = []
	if 'posts' in cache:
		posts = cache['posts']
	else:
		for x in islice(rf,15):
			posts.append(x)
			time.sleep(2*max(1,len(x) * 0.01))
		posts = [p for p in posts if bool(p['excerpt']) and bool(p['title'])]
		random.shuffle(posts)
		cache['posts'] = posts
		
	cache['ChatGPTWebsite'] = cache.get('ChatGPTWebsite',ChatGPTWebsite(driver,skip=False))
	gptWebsite = cache['ChatGPTWebsite'] 
	
	time.sleep(7)
	
	encodedChatLog: List[str] = []
	cache['encodedChatLog'] = encodedChatLog
	chatLog: List[Message] = []
	cache['chatLog'] = chatLog
	promptLog: List[str] = []
	cache['promptLog'] = promptLog
	dataLog: List[str] = []
	cache['dataLog'] = dataLog
	
	encoder = Encoder(data,dataLog,encodedChatLog,True)

	cache['aliceChat'] = cache.get('aliceChat', gptWebsite.getLandingNewChat())
	aliceChat = cache['aliceChat'] 
	if len(promptLog) == 0:
		# promptLog 0
		promptLog.append(prompts.start_conversation_with_post(alice,bob,posts[randint(0,len(posts)-1)]))
	
	if len(chatLog) == 0:
		# chatLog 0
		chatLog.append(Message(alice,aliceChat.ask(promptLog[0])))
	
	encoder(chatLog[0].text)

	if 'bobChat' not in cache or len(chatLog) == 1 or len(promptLog) == 1:
		# promptLog 1
		promptLog.append(prompts.startChatReplyPrompt(chatLog[0]))
		a,bobChat = aliceChat.askNew(promptLog[1])
		# chatLog 1
		chatLog.append(Message(bob,a))
		cache['bobChat'] = bobChat
	else:
		bobChat =cache['bobChat']

	# chatLog 2
	chatLog.append(Message(alice, aliceChat.ask(prompts.startChatReplyPrompt(chatLog[1]))))
	a = encoder(chatLog[2].text)
	
	cIdx = 2
	while True:
		if encoder.done():
			# chatLog cIdx+1
			chatLog.append(Message(alice, aliceChat.ask(prompts.endChatReplyPrompt(chatLog[cIdx]))))
			cIdx += 1
			a = encoder(chatLog[cIdx].text)
			return None

		# chatLog cIdx+1
		chatLog.append(Message(bob, bobChat.ask(prompts.inChatReplyPrompt(chatLog[cIdx],posts[randint(0,len(posts)-1)]))))
		cIdx += 1
		
		# switch
		
		# chatLog cIdx+1
		chatLog.append(Message(alice, aliceChat.ask(prompts.inChatReplyPrompt(chatLog[cIdx],posts[randint(0,len(posts)-1)]))))
		cIdx += 1
		a = encoder(chatLog[cIdx].text)


In [98]:
openChannelAndSend(data)

AttributeError: 'datetime.datetime' object has no attribute 'sleep'

In [ ]:
from json import dumps
from json import JSONEncoder
import pandas as pd
import shutil
import json
import subprocess
import os

def render(path):
  with open(path,'r') as f:
    cache = json.loads(f.read())
  normal_chat = json.dumps(cache['chatLog'])
  encoded_chat = []
  aliceChatLog: List[str] = cache['encodedChatLog']
  def f(i,m):
    m['text'] = aliceChatLog[0]
    aliceChatLog.remove(m['text'])
    return m
  aliceChatLog = list(f(i,m) for i, m in enumerate(cache['chatLog']) if m['person'] == 'alice nice')
  bobChatLog = list(m for m in cache['chatLog'] if m['person'] == 'bob wick')
  for i in range(max(len(aliceChatLog),len(bobChatLog))):
    if i < len(aliceChatLog):
      encoded_chat.append(aliceChatLog[i])
    if i < len(bobChatLog):
      encoded_chat.append(bobChatLog[i])
  encoded_chat = json.dumps(encoded_chat)
  with open('./chat.template.html','r') as f, open(f'{path}.html','w') as o:
    o.write(f.read().replace('$normal_chat',normal_chat)
    .replace('$encoded_chat',encoded_chat))
  file_path = os.path.abspath(f'{path}.html'.replace('/','\\'))
  print(file_path)
  try:
      subprocess.Popen(['start', '', file_path], shell=True)
      print(f"Opened {file_path} in the default application.")
  except Exception as e:
      print(f"Error: {e}")
def augment_stats():
  df = pd.read_csv('./stats.tsv',delimiter='\t')
  df['capacity'] = df['capacity'].apply(lambda x : round(100*x) if isinstance(x,float) else x)
  df.to_csv('./stats.tsv',sep='\t',index=False)
  dfd = df.describe()
  dfd['bits'] = dfd['bits'].apply(lambda a: int(round(a)))
  dfd['cover size'] = dfd['cover size'].apply(lambda a: int(round(a)))
  dfd['capacity'] = dfd['capacity'].apply(lambda a: int(round(a)))
  dfd.to_csv('./stats.stats.tsv',sep='\t')

class MessageEncoder(JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Message):
      return {
        'person': f"{obj.person.first_name} {obj.person.last_name}",
        'text': obj.text,
        'time': obj.time.strftime('%Y-%m-%d %H:%M:%S')
      }
    return super().default(obj)

def saveCache(cache=cache):
  id = uuid()
  dir_path = f'examples/{id}'
  os.mkdir(dir_path)

  with open(f'{dir_path}/cache.json','w') as f:
    f.write(dumps({
      'chatLog':cache['chatLog']
      , 'encodedChatLog': cache['encodedChatLog']
      , 'promptLog': cache['promptLog']
      , 'dataLog': cache['dataLog']
      , 'data': cache['data']
      },cls=MessageEncoder))
  augment_stats()
  shutil.move('stats.tsv', dir_path+'/stats.tsv')
  shutil.move('stats.stats.tsv', dir_path+'/stats.stats.tsv')
  shutil.move('step.log', dir_path+'/step.log')
  shutil.move('pie.log', dir_path+'/pie.log')
  render(f'{dir_path}/cache.json')

saveCache()
